<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ce1df608d583a027380eb483d00c6432900565e16231e3675d24776a151b74a5
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-30 14:41:59
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: 8.49 K (0.06%)
Current PnL: -23.04 L (-15.5%)
CY Booked + Current PnL: -11.60 L (-7.8%)
-------------------
Total profit:  1.13 L
Total loss:  -24.17 L
-------------------
Total Booked + Current PnL: 15.38 L (12.64%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.38%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.91 L (65.11%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 6.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.26,112816.0,12312.0,16798.0,4.39,12.25,14.89,28.96,79.0,0.73,0.83,50.38,MH,ATH,METALS
77,TTKPRESTIG,770.00,92.53,35.0,M-SC,2.43,82923.0,-17854.0,17945.0,-0.31,-17.72,21.64,0.09,245.0,-0.99,0.61,8.94,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,3.63,189560.0,24890.0,18444.0,0.29,15.12,9.73,26.31,89.0,1.35,1.39,24.30,X40N,NTT,AC
42,ITC,452.00,-39.19,42.0,H-LC,1.96,196382.0,-3756.0,24646.0,-1.39,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG
84,VOLTAS,1530.00,-2.48,45.0,H-MC,2.86,204540.0,12798.0,24954.0,1.75,6.67,12.20,19.69,99.0,0.51,1.50,13.93,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1233.33,68.0,M-SC,14.23,181337.0,194.0,45280.0,5.88,0.11,24.97,25.10,235.0,0.00,1.33,45.32,XY24,NTT,HEALTHCARE
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.26,112816.0,12312.0,16798.0,4.39,12.25,14.89,28.96,79.0,0.73,0.83,50.38,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,64.0,H-MC,8.92,189251.0,9443.0,102971.0,3.60,5.25,54.41,62.52,88.0,0.09,1.39,41.13,XR,NTT,BANKS
79,UNIONBANK,163.00,-13.18,58.0,M-LC,7.14,157918.0,17078.0,27415.0,3.59,12.13,17.36,31.59,66.0,0.62,1.16,41.83,XY24,NTT,BANKS
32,HINDZINC,730.22,28.52,65.0,M-LC,3.43,209452.0,4376.0,108203.0,3.29,2.13,51.66,54.89,52.0,0.04,1.53,26.74,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-47.73,24.0,M-SC,1.21,88395.0,-3104.0,81854.0,-4.38,-3.39,92.60,86.07,223.0,-0.04,0.65,28.37,XR,NTT,DURABLES
55,RAJOOENG,143.10,-45.85,20.0,H-SC,14.28,82000.0,-20500.0,61098.0,-4.12,-20.00,74.51,39.61,114.0,-0.34,0.60,0.00,AR,ATH,MISC
81,VAIBHAVGBL,521.00,46.01,42.0,H-SC,5.82,126748.0,-56027.0,167092.0,-3.33,-30.65,131.83,60.77,125.0,-0.34,0.93,13.52,XR,NTT,JEWELLERY
38,INDIGOPNTS,1408.00,99.78,28.0,M-SC,1.55,131142.0,-43417.0,43447.0,-3.03,-24.87,33.13,0.02,221.0,-1.00,0.96,14.07,OX40N,NTT,PAINTS
3,ALKYLAMINE,4546.37,-11.57,29.0,H-SC,10.70,82298.0,-18665.0,113193.0,-2.86,-18.49,137.54,93.63,148.0,-0.16,0.60,22.58,SR,ATH,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1233.33,68.0,M-SC,14.23,181337.0,194.0,45280.0,5.88,0.11,24.97,25.10,235.0,0.00,1.33,45.32,XY24,NTT,HEALTHCARE
37,INDIAMART,4850.92,-55.39,24.0,H-SC,9.98,124465.0,1129.0,132630.0,-0.78,0.92,106.56,108.45,119.0,0.01,0.91,23.47,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,98.37,58.0,H-SC,10.85,125307.0,-17172.0,36690.0,0.72,-12.05,29.28,13.70,163.0,-0.47,0.92,47.90,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,92.53,35.0,M-SC,2.43,82923.0,-17854.0,17945.0,-0.31,-17.72,21.64,0.09,245.0,-0.99,0.61,8.94,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,52.0,H-SC,1.54,221598.0,-48069.0,84407.0,-0.15,-17.83,38.09,13.47,138.0,-0.57,1.62,12.08,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,37.0,H-SC,1.59,141462.0,-34441.0,76460.0,-0.13,-19.58,54.05,23.89,149.0,-0.45,1.04,22.17,OX40N,NTT,CERAMICS
66,SIS,528.0,2092.75,43.0,H-SC,4.03,88498.0,-22534.0,46143.0,0.54,-20.30,52.14,21.26,156.0,-0.49,0.65,19.57,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1233.33,68.0,M-SC,14.23,181337.0,194.0,45280.0,5.88,0.11,24.97,25.10,235.0,0.00,1.33,45.32,XY24,NTT,HEALTHCARE
60,SAIL,228.00,46.75,58.0,M-MC,11.22,231230.0,6268.0,161375.0,1.95,2.79,69.79,74.53,192.0,0.04,1.69,35.94,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,65.0,M-LC,3.43,209452.0,4376.0,108203.0,3.29,2.13,51.66,54.89,52.0,0.04,1.53,26.74,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,3.63,189560.0,24890.0,18444.0,0.29,15.12,9.73,26.31,89.0,1.35,1.39,24.30,X40N,NTT,AC
84,VOLTAS,1530.00,-2.48,45.0,H-MC,2.86,204540.0,12798.0,24954.0,1.75,6.67,12.20,19.69,99.0,0.51,1.50,13.93,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.26,112816.0,12312.0,16798.0,4.39,12.25,14.89,28.96,79.0,0.73,0.83,50.38,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,64.0,H-MC,8.92,189251.0,9443.0,102971.0,3.60,5.25,54.41,62.52,88.0,0.09,1.39,41.13,XR,NTT,BANKS
32,HINDZINC,730.22,28.52,65.0,M-LC,3.43,209452.0,4376.0,108203.0,3.29,2.13,51.66,54.89,52.0,0.04,1.53,26.74,X5K,ATH,METALS
37,INDIAMART,4850.92,-55.39,24.0,H-SC,9.98,124465.0,1129.0,132630.0,-0.78,0.92,106.56,108.45,119.0,0.01,0.91,23.47,AR,ATH,MISC
86,WIPRO,420.00,-15.15,40.0,M-LC,5.58,149172.0,-1773.0,111223.0,0.40,-1.17,74.56,72.51,53.0,-0.02,1.09,4.78,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-6.98,26.0,H-LC,13.02,252754.0,-14611.0,80072.0,-0.10,-5.46,31.68,24.48,48.0,-0.18,1.85,46.50,XY24,BTT,FINANCE
74,TCS,4476.75,-31.58,28.0,H-LC,12.29,272139.0,-73817.0,148670.0,-0.03,-21.34,54.63,21.64,1.0,-0.50,1.99,0.00,X40,ATH,IT
5,ASIANPAINT,3465.66,-18.03,30.0,H-LC,6.16,206914.0,-44854.0,98057.0,0.37,-17.82,47.39,21.13,27.0,-0.46,1.52,11.30,X40,ATH,PAINTS
83,VBL,671.64,-21.55,32.0,H-LC,8.84,282379.0,-33463.0,146131.0,-2.01,-10.59,51.75,35.67,5.0,-0.23,2.07,1.79,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.16,218167.0,-23757.0,87027.0,0.16,-9.82,39.89,26.15,8.0,-0.27,1.60,3.10,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.65,44.0,H-LC,1.63,231278.0,-14873.0,94084.0,-0.36,-6.04,40.68,32.18,16.0,-0.16,1.69,27.82,X200,ATH,IT
42,ITC,452.0,-39.19,42.0,H-LC,1.96,196382.0,-3756.0,24646.0,-1.39,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.61,44.0,H-LC,2.39,156450.0,-29645.0,77130.0,1.06,-15.93,49.30,25.51,15.0,-0.38,1.15,15.29,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.79,41.0,H-LC,3.77,213470.0,3104.0,25680.0,-0.32,1.48,12.03,13.68,37.0,0.12,1.56,18.24,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.87,38.0,H-LC,3.99,205418.0,918.0,40488.0,0.77,0.45,19.71,20.25,10.0,0.02,1.50,9.22,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,28.0,H-LC,12.29,272139.0,-73817.0,148670.0,-0.03,-21.34,54.63,21.64,1.0,-0.50,1.99,0.00,X40,ATH,IT
83,VBL,671.64,-21.55,32.0,H-LC,8.84,282379.0,-33463.0,146131.0,-2.01,-10.59,51.75,35.67,5.0,-0.23,2.07,1.79,X40N,ATH,FMCG
80,UNITDSPR,1644.00,-14.80,43.0,H-LC,7.21,223239.0,-10905.0,59538.0,-0.81,-4.66,26.67,20.77,86.0,-0.18,1.64,1.94,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.16,218167.0,-23757.0,87027.0,0.16,-9.82,39.89,26.15,8.0,-0.27,1.60,3.10,X40,ATH,IT
42,ITC,452.00,-39.19,42.0,H-LC,1.96,196382.0,-3756.0,24646.0,-1.39,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,44.0,H-LC,2.39,156450.0,-29645.0,77130.0,1.06,-15.93,49.30,25.51,15.0,-0.38,1.15,15.29,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,42.0,H-LC,1.96,196382.0,-3756.0,24646.0,-1.39,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG
19,CIPLA,1795.00,-22.87,38.0,H-LC,3.99,205418.0,918.0,40488.0,0.77,0.45,19.71,20.25,10.0,0.02,1.50,9.22,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.03,30.0,H-LC,6.16,206914.0,-44854.0,98057.0,0.37,-17.82,47.39,21.13,27.0,-0.46,1.52,11.30,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.79,41.0,H-LC,3.77,213470.0,3104.0,25680.0,-0.32,1.48,12.03,13.68,37.0,0.12,1.56,18.24,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,28.0,H-LC,12.29,272139.0,-73817.0,148670.0,-0.03,-21.34,54.63,21.64,1.0,-0.50,1.99,0.00,X40,ATH,IT
40,INFY,2275.00,-21.03,38.0,H-LC,8.21,307380.0,-5508.0,177205.0,0.09,-1.76,57.65,54.87,3.0,-0.03,2.25,4.77,X40,BTT,IT
42,ITC,452.00,-39.19,42.0,H-LC,1.96,196382.0,-3756.0,24646.0,-1.39,-1.88,12.55,10.44,4.0,-0.15,1.44,3.58,X40,NTT,FMCG
83,VBL,671.64,-21.55,32.0,H-LC,8.84,282379.0,-33463.0,146131.0,-2.01,-10.59,51.75,35.67,5.0,-0.23,2.07,1.79,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,45.0,H-LC,7.69,249365.0,-12254.0,131465.0,0.42,-4.68,52.72,45.57,7.0,-0.09,1.83,6.11,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,40.33,79450.0,-34099.0,74151.0,2.35,-30.03,93.33,35.27,268.0,-0.46,0.58,94.45,XR,NTT,HOTELS
49,MASFIN,398.61,-21.41,34.0,H-SC,12.25,89430.0,-8550.0,30156.0,-1.50,-8.73,33.72,22.05,152.0,-0.28,0.66,29.99,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-181.82,75.0,M-SC,7.09,94140.0,-8070.0,101464.0,1.70,-7.90,107.78,91.37,208.0,-0.08,0.69,52.82,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.26,112816.0,12312.0,16798.0,4.39,12.25,14.89,28.96,79.0,0.73,0.83,50.38,MH,ATH,METALS
78,UJJIVANSFB,60.00,98.37,58.0,H-SC,10.85,125307.0,-17172.0,36690.0,0.72,-12.05,29.28,13.70,163.0,-0.47,0.92,47.90,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1233.33,68.0,M-SC,14.23,181337.0,194.0,45280.0,5.88,0.11,24.97,25.10,235.0,0.00,1.33,45.32,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-181.82,75.0,M-SC,7.09,94140.0,-8070.0,101464.0,1.70,-7.90,107.78,91.37,208.0,-0.08,0.69,52.82,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.26,112816.0,12312.0,16798.0,4.39,12.25,14.89,28.96,79.0,0.73,0.83,50.38,MH,ATH,METALS
60,SAIL,228.00,46.75,58.0,M-MC,11.22,231230.0,6268.0,161375.0,1.95,2.79,69.79,74.53,192.0,0.04,1.69,35.94,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,65.0,M-LC,3.43,209452.0,4376.0,108203.0,3.29,2.13,51.66,54.89,52.0,0.04,1.53,26.74,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.21
1,25,43.71
2,50,75.10


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.66
LC    32.55
MC    23.82
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.39
X40      14.79
X40N     11.37
XR       11.09
XY25     10.25
AR        8.20
OX40N     7.89
X200      1.69
MH        1.64
X5K       1.53
SR        1.19
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.15
H-LC    25.90
H-MC    20.85
M-SC    13.06
M-LC     5.59
M-MC     2.66
L-SC     1.45
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.08,-6.86,42.13
IT,12.69,-21.36,87.53
FINANCE,10.12,-18.92,70.80
MISC,6.92,-21.54,89.10
BANKS,6.36,-15.12,75.68
PAINTS,5.62,-20.35,38.29
ELECTRICAL,5.49,-10.03,48.63
INSURANCE,3.78,-7.29,47.40
AC,3.60,1.38,15.14


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3229277.0,22
XR,1396812.0,14
AR,1214692.0,9
X40,831143.0,10
X40N,653774.0,9
OX40N,571336.0,10
XY25,441686.0,7
SR,258992.0,2
X5K,108203.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3817508.0,29
M-SC,1462674.0,17
H-LC,1284876.0,15
H-MC,1262939.0,15
M-LC,386548.0,4
M-MC,313431.0,2
L-SC,262040.0,3
L-MC,60611.0,1
L-LC,40023.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1248860.0      6
M-SC       XY24       836330.0      7
H-SC       AR         834996.0      5
           XR         818146.0      7
H-MC       XY24       583462.0      4
H-LC       X40        535605.0      5
H-SC       X40N       320951.0      4
M-MC       XY24       313431.0      2
H-SC       OX40N      261710.0      4
           SR         258992.0      2
H-LC       X40N       246157.0      3
H-MC       X40        218551.0      4
H-LC       AR         208595.0      2
H-MC       XY25       179836.0      2
L-SC       XR         172950.0      2
M-SC       AR         171101.0      2
M-LC       XY24       167122.0      2
M-SC       OX40N      145881.0      4
           XR         130911.0      2
H-LC       XY25       120363.0      3
M-LC       XR         111223.0      1
           X5K        108203.0      1
H-MC       XR         102971.0      1
M-SC       XY25       101464.0      1
H-LC       X200        94084.0      1
L-SC       OX40N       89090.0      1
H-MC       X40N        86666.0      2
H-LC       XY24        80072.0      1
M-SC       X40         76987.0      1
H-MC       OX40N       74655.0      1
H-SC       MH          73853.0      1
L-MC       XR          60611.0      1
L-LC       XY25        40023.0      1
H-MC       MH          16798.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
